In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/datos_delitos.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 1: invalid continuation byte

# Errorde Encoding

Aquí tenemos un error muy común que suele ocurrir cuando un archivo se guarda en una computadora con cierto "encoding".

Un encoding, a grandes rasgos, es una tabla que relaciona un número con un carácter. Por ejemplo, en la tabla ASCII, el número 65 corresponde a la letra "A".

Si el archivo que estamos leyendo fue guardado con un encoding distinto al que pandas espera, nos arrojará un error.

Para solucionar esto, podemos utilizar el parámetro encoding de la función pd.read_csv() y especificar el encoding correcto.

¿Pero cómo sabemos con qué encoding cuenta el archivo?

La realidad es que la gran mayoría de los archivos los encontrarán en encoding utf-8 y Pandas no va a dar ningún error. Aquí estamos teniendo este problema porque la computadora que utilizan para generar este archivo uso un encoding diferente a utf-8.

En México, por lo general, si un archivo no está en utf-8, lo más seguro es que esté en ISO-8859-1 o latin1.

Intentemos cargar el archivo especificando el encoding ISO-8859-1

In [4]:
df = pd.read_csv('data/datos_delitos.csv', encoding='ISO-8859-1')
df.head()

,Año,Clave_Ent,Entidad,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,3,0,2,1,1,1,2,1,2,2,2,1.0
1,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,1,1,0,0,0,1,0,1,0,0,0,1.0
2,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,0,0,2,2,3,2,0,1,2,0,0,0.0
3,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,2,0,0,1,0,0,0,0,0,0,0,0.0
4,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio culposo,Con arma de fuego,0,0,0,0,1,0,0,0,0,0,0,0.0


# ¿Existe alguna forma de verificar el encoding de un archivo sin tener que estar adivinando?

In [4]:
import chardet

def detect_encoding_safe(file_path, n_bytes=100_000):
    with open(file_path, 'rb') as f:
        rawdata = f.read(n_bytes)
    return chardet.detect(rawdata)

file_path = './data/datos_delitos.csv'
encoding_info = detect_encoding_safe(file_path)

encoding_info


{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}

- Le pedi a ChatGPT que modificara el codigo del libro, ya que eran tantos datos que se trababa al querer leer todos. Este codigo solo lee 100 kb

- Tambien tuve que instalar la libreria chardet

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34496 entries, 0 to 34495
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Año                     34496 non-null  int64  
 1   Clave_Ent               34496 non-null  int64  
 2   Entidad                 34496 non-null  object 
 3   Bien jurídico afectado  34496 non-null  object 
 4   Tipo de delito          34496 non-null  object 
 5   Subtipo de delito       34496 non-null  object 
 6   Modalidad               34496 non-null  object 
 7   Enero                   34496 non-null  int64  
 8   Febrero                 34496 non-null  int64  
 9   Marzo                   34496 non-null  int64  
 10  Abril                   34496 non-null  int64  
 11  Mayo                    34496 non-null  int64  
 12  Junio                   34496 non-null  int64  
 13  Julio                   34496 non-null  int64  
 14  Agosto                  34496 non-null

Bien. Están muy bien los datos. Sin embargo, tenemos un problema con el que es muy común encontrarnos.

Resulta que el formato en el que está el archivo es fácil entender por seres humanos:

| Año      | Entidad  | Enero    | Febrero  | Mes X    |
|----------|----------|----------|----------|----------|
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |

Vemos que tenemos los meses como encabezados. Es decir, el archivo está tratando cada mes como si fuera una variable.

Nosotros como científicos de datos estamos más interesados en conjuntos de datos que no estén en este formato de "resumen" o "tabla dinámica". Para nosotros, lo ideal sería que cada mes fuera simplemente una observación más en nuestro conjunto de datos. Es decir, queremos transformar la tabla de arriba en:

| Año      | Entidad  | Mes        |
|----------|----------|------------|
|   ...    |   ...    |   Enero    |
|   ...    |   ...    |   Febrero  |
|   ...    |   ...    |   Marzo    |
|   ...    |   ...    |   Abril    |
|   ...    |   ...    |   Mes X    |

# Hora de limpiar datos

Haremos las siguientes “limpiezas”:
- Transformar los nombres de las columnas para que no tengan caracteres especiales y estén siempre en minúsculas
- Convertir el dataset a un formato de datos "largo"

# Limpiar nombres de columnas

Recordemos que df.columns nos regresa una especie de lista que contiene los nombres de las columnas:

In [9]:
for col in df.columns:
    print(col)

Año
Clave_Ent
Entidad
Bien jurídico afectado
Tipo de delito
Subtipo de delito
Modalidad
Enero
Febrero
Marzo
Abril
Mayo
Junio
Julio
Agosto
Septiembre
Octubre
Noviembre
Diciembre


# DEF para limiar columnas

In [10]:
def limpiar_columnas(df):
    columnas_limpias = []
    for col in df.columns:
        # convertir a minusculas, reemplazar espacios por guiones bajos y eliminar caracteres especiales
        col = col.lower().replace(" ", "_").replace("ñ", "ni").replace(".", "").replace("á", "a").replace("é", "e").replace("í","i").replace("ó", "o").replace("ú", "u")
        columnas_limpias.append(col)
    
    df.columns = columnas_limpias
    
    return df

- Esto solamente se puede hacer si la nueva lista columnas_limpias tiene la misma cantidad de elementos que df.columns.

In [12]:
df = limpiar_columnas(df) #Aqui uso mi funcion guardandola sobre mi df origininal
df.head(3)

,anio,clave_ent,entidad,bien_juridico_afectado,tipo_de_delito,subtipo_de_delito,modalidad,enero,febrero,marzo,abril,mayo,junio,julio,agosto,septiembre,octubre,noviembre,diciembre
0,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,3,0,2,1,1,1,2,1,2,2,2,1.0
1,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,1,1,0,0,0,1,0,1,0,0,0,1.0
2,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,0,0,2,2,3,2,0,1,2,0,0,0.0


# QUITAR COLUMNAS

In [13]:
df = df[['anio', 'clave_ent', 'entidad', 'tipo_de_delito', 'subtipo_de_delito', 'modalidad','enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']]
df.head(2)

,anio,clave_ent,entidad,tipo_de_delito,subtipo_de_delito,modalidad,enero,febrero,marzo,abril,mayo,junio,julio,agosto,septiembre,octubre,noviembre,diciembre
0,2015,1,Aguascalientes,Homicidio,Homicidio doloso,Con arma de fuego,3,0,2,1,1,1,2,1,2,2,2,1.0
1,2015,1,Aguascalientes,Homicidio,Homicidio doloso,Con arma blanca,1,1,0,0,0,1,0,1,0,0,0,1.0


# Formato largo de datos

Ahora, usaremos el método melt para convertir las columnas a observaciones.

Queremos convervar las coumnas:

- anio
- clave_ent
- entidad
- tipo_de_delito
- subtipo_de_delito
- modalidad

El resto de las columnas las vamos a juntar en una nueva columna llamada "nombre_mes" y sus valores los vamos a sumar en otra llamada "frecuencia".

In [14]:
print("Shape: ", df.shape)

Shape:  (34496, 18)


# Ahora creemos un nuevo dataframe usando melt

In [15]:
datos_long = df.melt(
  id_vars=['anio', 'clave_ent', 'entidad','tipo_de_delito', 'subtipo_de_delito', 'modalidad'],
  var_name='nombre_mes',
  value_name='frecuencia'
)

Y veamos cómo cambia el Shape

In [16]:
print("Shape: ", datos_long.shape)

Shape:  (413952, 8)


In [17]:
datos_long.head(5)

,anio,clave_ent,entidad,tipo_de_delito,subtipo_de_delito,modalidad,nombre_mes,frecuencia
0,2015,1,Aguascalientes,Homicidio,Homicidio doloso,Con arma de fuego,enero,3.0
1,2015,1,Aguascalientes,Homicidio,Homicidio doloso,Con arma blanca,enero,1.0
2,2015,1,Aguascalientes,Homicidio,Homicidio doloso,Con otro elemento,enero,0.0
3,2015,1,Aguascalientes,Homicidio,Homicidio doloso,No especificado,enero,2.0
4,2015,1,Aguascalientes,Homicidio,Homicidio culposo,Con arma de fuego,enero,0.0


In [18]:
datos_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413952 entries, 0 to 413951
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   anio               413952 non-null  int64  
 1   clave_ent          413952 non-null  int64  
 2   entidad            413952 non-null  object 
 3   tipo_de_delito     413952 non-null  object 
 4   subtipo_de_delito  413952 non-null  object 
 5   modalidad          413952 non-null  object 
 6   nombre_mes         413952 non-null  object 
 7   frecuencia         410816 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 25.3+ MB


Supongamos que para este análisis, no nos importan los niveles subtipo de delito y modalidad. O sea, no queremos tener la distinción entre homicidios dolosos y culposos (claro está que estos son muy diferentes, pero ignoremos esta diferencia con el fin de simplificar este ejercicio).

Vamos a agrupar nuestro dataframe por anio, clave_ent, entidad, tipo_de_delito y nombre_mes. Esto hará que todos los tipos de homicidios se sumen al tipo "homicidio" o todos los tipos de robo de vehículo (con o sin violencia) se sumen a "robo de vehículo".

In [19]:
datos_long = datos_long.groupby(['anio', 'clave_ent', 'entidad', 'tipo_de_delito', 'nombre_mes'])['frecuencia'].sum().reset_index()
datos_long[datos_long.tipo_de_delito == 'Robo'].sample(15)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia
38312,2017,16,Michoacán de Ocampo,Robo,mayo,1525.0
40709,2017,21,Puebla,Robo,julio,2531.0
108394,2022,2,Baja California,Robo,octubre,2790.0
154945,2025,3,Baja California Sur,Robo,agosto,459.0
41191,2017,22,Querétaro,Robo,marzo,2259.0
66155,2019,10,Durango,Robo,septiembre,871.0
66634,2019,11,Guanajuato,Robo,octubre,3684.0
160705,2025,15,México,Robo,agosto,7769.0
122315,2022,31,Yucatán,Robo,septiembre,49.0
8075,2015,17,Morelos,Robo,septiembre,1765.0


Lo acabamos de hacer es “eliminar” las dimensiones de subtipo de delito y modalidad.

Mostremos todos los estados y su respectiva clave

In [20]:
datos_long[['clave_ent', 'entidad']].drop_duplicates().sort_values('entidad')

,clave_ent,entidad
0,1,Aguascalientes
480,2,Baja California
960,3,Baja California Sur
1440,4,Campeche
2880,7,Chiapas
3360,8,Chihuahua
3840,9,Ciudad de México
1920,5,Coahuila de Zaragoza
2400,6,Colima
4320,10,Durango


Ahora Veamos todos los datos de delitos de una entidad en específico. Por ejemplo, Nuevo león

In [21]:
datos_long[datos_long['clave_ent'] == 19].sample(15)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia
24130,2016,19,Nuevo León,Despojo,octubre,71.0
162639,2025,19,Nuevo León,Secuestro,enero,3.0
54787,2018,19,Nuevo León,Amenazas,marzo,374.0
146926,2024,19,Nuevo León,Acoso sexual,octubre,77.0
116507,2022,19,Nuevo León,Otros delitos que atentan contra la libertad p...,septiembre,234.0
39539,2017,19,Nuevo León,Falsedad,septiembre,23.0
116417,2022,19,Nuevo León,Incumplimiento de obligaciones de asistencia f...,julio,44.0
100850,2021,19,Nuevo León,Allanamiento de morada,diciembre,50.0
162411,2025,19,Nuevo León,Falsedad,enero,19.0
101063,2021,19,Nuevo León,Incumplimiento de obligaciones de asistencia f...,septiembre,51.0


In [40]:
datos_long[(datos_long['clave_ent'] == 30) & (datos_long['tipo_de_delito'] == 'Feminicidio')].sample(30)




,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia
90921,2020,30,Veracruz de Ignacio de la Llave,Feminicidio,noviembre,6.0
90918,2020,30,Veracruz de Ignacio de la Llave,Feminicidio,junio,8.0
29477,2016,30,Veracruz de Ignacio de la Llave,Feminicidio,julio,4.0
90919,2020,30,Veracruz de Ignacio de la Llave,Feminicidio,marzo,7.0
90914,2020,30,Veracruz de Ignacio de la Llave,Feminicidio,diciembre,5.0
137002,2023,30,Veracruz de Ignacio de la Llave,Feminicidio,octubre,1.0
106279,2021,30,Veracruz de Ignacio de la Llave,Feminicidio,marzo,7.0
152357,2024,30,Veracruz de Ignacio de la Llave,Feminicidio,julio,4.0
14112,2015,30,Veracruz de Ignacio de la Llave,Feminicidio,abril,4.0
152360,2024,30,Veracruz de Ignacio de la Llave,Feminicidio,mayo,9.0


# Valores de fechas

Finalmente, queremos tener una columna "fecha". Actualmente tenemos el año y el nombre del mes, pero no tenemos como tal una columna que tenga un tipo de dato fecha. Eso hace que filtrar por fecha sea complicado.

Por ejemplo, si queremos conocer todos los homicidios de Oaxaca en enero 2024, haríamos lo siguiente:

In [41]:
datos_long[
    (datos_long['clave_ent'] == 20) &
    (datos_long['tipo_de_delito'] == 'Homicidio') &
    (datos_long['anio'] == 2024) &
    (datos_long['nombre_mes'] == 'enero') 
]

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia
147579,2024,20,Oaxaca,Homicidio,enero,154.0


Veremos que Pandas hace muy sencillo el manejo de fechas.

Primero que nada, creemos nuea nueva columna. La forma más fácil de crear una nueva columna en un dataframe de pandas es de la misma forma que agregamos nuevos valores a un diccionario:

In [42]:
df["nueva_columna"] = "valor"

Esta línea anterior, crearía una nueva columna llamada nueva_columna, y se le asignaría el string “valor” a todos los valores de esta nueva serie.

Haremos algo un poco diferente en este caso:

In [43]:
# Diccionario de ayuda para convertir
meses = {
    "enero": 1,
    "febrero": 2,
    "marzo": 3,
    "abril": 4,
    "mayo": 5,
    "junio": 6,
    "julio": 7,
    "agosto": 8,
    "septiembre": 9,
    "octubre": 10,
    "noviembre": 11,
    "diciembre": 12
}

datos_long['mes'] = datos_long['nombre_mes'].map(meses)
datos_long.sample(5)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia,mes
26664,2016,24,San Luis Potosí,Lesiones,abril,330.0,4
149972,2024,25,Sinaloa,Fraude,mayo,101.0,5
36631,2017,13,Hidalgo,Evasión de presos,marzo,0.0,3
101272,2021,19,Nuevo León,Violencia familiar,febrero,1257.0,2
157028,2025,8,Chihuahua,Amenazas,mayo,281.0,5


Ahora tenemos una nueva columna llamada “mes” que contiene un simple entero que representa el número del mes en cuestión. Ahora, utilizaremos funciones específicas de datetime de pandas. Nos apalancaremos de las capacidades de pandas y numpy para manipular datos de manera vectorizada:

In [44]:
datos_long['fecha'] = pd.to_datetime(
  datos_long['anio'].astype(str) + datos_long['mes'].astype(str),
  format='%Y%m'
)
datos_long.sample(5)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia,mes,fecha
74666,2019,28,Tamaulipas,Lesiones,diciembre,248.0,12,2019-12-01
96044,2021,9,Ciudad de México,Acoso sexual,mayo,120.0,5,2021-05-01
37474,2017,15,México,Abuso sexual,octubre,166.0,10,2017-10-01
132192,2023,20,Oaxaca,Feminicidio,abril,2.0,4,2023-04-01
27713,2016,26,Sonora,Otros delitos que atentan contra la libertad y...,julio,12.0,7,2016-07-01


Ahora que tenemos una columna de tipo de dato datetime, podemos hacer filtros de fechas mucho más fácilmente:

In [45]:
datos_long[
    (datos_long.tipo_de_delito == "Homicidio") &
    (datos_long.clave_ent == 20) &
    (datos_long.fecha >= '2024-01-01')
].sort_values('fecha')

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia,mes,fecha
147579,2024,20,Oaxaca,Homicidio,enero,154.0,1,2024-01-01
147580,2024,20,Oaxaca,Homicidio,febrero,166.0,2,2024-02-01
147583,2024,20,Oaxaca,Homicidio,marzo,184.0,3,2024-03-01
147576,2024,20,Oaxaca,Homicidio,abril,200.0,4,2024-04-01
147584,2024,20,Oaxaca,Homicidio,mayo,223.0,5,2024-05-01
147582,2024,20,Oaxaca,Homicidio,junio,195.0,6,2024-06-01
147581,2024,20,Oaxaca,Homicidio,julio,201.0,7,2024-07-01
147577,2024,20,Oaxaca,Homicidio,agosto,186.0,8,2024-08-01
147587,2024,20,Oaxaca,Homicidio,septiembre,190.0,9,2024-09-01
147586,2024,20,Oaxaca,Homicidio,octubre,185.0,10,2024-10-01


- Obtuvimos muy fácilmente todos los homicidios en Oaxaca desde enero 2024 a la fecha.

Ya que tenemos muestros datos bien estructurados, los podemos guardar en nuestra computadora. Los guardaremos con el nombre "delitos.csv" en la carpeta “data”.

Estos los utilizaremos más adelante, entonces asegúrate de guardarlos.

In [46]:
datos_finales.to_csv('data/delitos.csv', index=False)

NameError: name 'datos_finales' is not defined

In [48]:
datos_finales = datos_long
datos_finales.to_csv('data/delitos.csv', index=False)

